In [1]:
pip install qiskit cirq amazon-braket-sdk cryptography pynacl pycryptodome numpy scipy matplotlib pandas tqdm jupyter seaborn ipywidgets pqcrypto websockets fastapi uvicorn


  Using cached cirq-1.6.1-py3-none-any.whl.metadata (16 kB)
  Using cached amazon_braket_sdk-1.106.1-py3-none-any.whl.metadata (14 kB)
  Using cached cirq_aqt-1.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached cirq_core-1.6.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached cirq_google-1.6.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached cirq_ionq-1.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached cirq_pasqal-1.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached cirq_web-1.6.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached duet-0.2.9-py3-none-any.whl.metadata (2.3 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached typedunits-0.0.1-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached grpcio_status-1.76.

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.41.5 which is incompatible.
dask 2025.2.0 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
distributed 2025.2.0 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.


In [1]:
# Check if pycryptodome is installed
try:
    from Crypto.Cipher import AES, DES
    from Crypto.Util.Padding import pad, unpad
    from Crypto.Random import get_random_bytes
    print("Crypto library is installed ✅")
except ImportError:
    print("Crypto library is NOT installed ❌")


Crypto library is installed ✅


In [3]:
# =============================
# HYBRID QUANTUM-CLASSICAL SOCKET SIMULATION
# =============================

import socket
import threading
import time
from Crypto.Cipher import AES, DES
from Crypto.Util.Padding import pad, unpad
import numpy as np

# =============================
# 1. Quantum-inspired Superdense Key Simulation
# =============================
def generate_quantum_key(length=16):
    """
    Simulate a quantum superdense coding key.
    :param length: 16 bytes for AES, 8 bytes for DES
    """
    # Simulate "entangled qubits" as random bits
    qubits = np.random.randint(0, 2, size=length*8)  # 8 bits per byte
    key_bytes = bytearray()
    for i in range(0, len(qubits), 8):
        byte = 0
        for j in range(8):
            byte = (byte << 1) | qubits[i+j]
        key_bytes.append(byte)
    return bytes(key_bytes)

# =============================
# 2. Encryption / Decryption
# =============================
def encrypt_message_AES(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    return cipher.iv + ct_bytes

def decrypt_message_AES(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def encrypt_message_DES(message, key):
    cipher = DES.new(key, DES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), DES.block_size))
    return cipher.iv + ct_bytes

def decrypt_message_DES(ciphertext, key):
    iv = ciphertext[:8]
    ct = ciphertext[8:]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), DES.block_size)
    return pt.decode()

# =============================
# 3. Server Code
# =============================
def server(port=12345, use_AES=True):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', port))
    s.listen(1)
    print(f"[Server] Listening on port {port}...")
    
    conn, addr = s.accept()
    print(f"[Server] Connection from {addr}")

    # Quantum-inspired key generation
    key_length = 16 if use_AES else 8
    key = generate_quantum_key(key_length)

    # Send key to client (simulated securely via quantum channel)
    conn.send(key)

    # Receive encrypted message
    encrypted_msg = conn.recv(4096)
    if use_AES:
        message = decrypt_message_AES(encrypted_msg, key)
    else:
        message = decrypt_message_DES(encrypted_msg, key)

    print(f"[Server] Received (decrypted): {message}")
    conn.close()

# =============================
# 4. Client Code
# =============================
def client(port=12345, message="Hello Quantum World!", use_AES=True):
    time.sleep(1)  # ensure server is ready
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(('localhost', port))

    # Receive key (simulated via quantum channel)
    key_length = 16 if use_AES else 8
    key = s.recv(key_length)

    # Encrypt message using AES or DES
    if use_AES:
        encrypted_msg = encrypt_message_AES(message, key)
    else:
        encrypted_msg = encrypt_message_DES(message, key)

    s.send(encrypted_msg)
    s.close()
    print(f"[Client] Sent message (encrypted).")

# =============================
# 5. Run Simulation in Jupyter
# =============================
use_AES = True  # Change to False for DES

server_thread = threading.Thread(target=server, args=(12345, use_AES))
client_thread = threading.Thread(target=client, args=(12345, "Hello Quantum Socket!", use_AES))

server_thread.start()
client_thread.start()

server_thread.join()
client_thread.join()


[Server] Listening on port 12345...
[Server] Connection from ('127.0.0.1', 63812)
[Client] Sent message (encrypted).
[Server] Received (decrypted): Hello Quantum Socket!


In [4]:
# =============================
# HYBRID QUANTUM-CLASSICAL SOCKET SIMULATION (Bidirectional)
# =============================

import socket
import threading
import time
from Crypto.Cipher import AES, DES
from Crypto.Util.Padding import pad, unpad
import numpy as np

# =============================
# 1. Quantum-inspired Superdense Key Simulation
# =============================
def generate_quantum_key(length=16):
    qubits = np.random.randint(0, 2, size=length*8)  # 8 bits per byte
    key_bytes = bytearray()
    for i in range(0, len(qubits), 8):
        byte = 0
        for j in range(8):
            byte = (byte << 1) | qubits[i+j]
        key_bytes.append(byte)
    return bytes(key_bytes)

# =============================
# 2. Encryption / Decryption
# =============================
def encrypt_message_AES(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    return cipher.iv + ct_bytes

def decrypt_message_AES(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def encrypt_message_DES(message, key):
    cipher = DES.new(key, DES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), DES.block_size))
    return cipher.iv + ct_bytes

def decrypt_message_DES(ciphertext, key):
    iv = ciphertext[:8]
    ct = ciphertext[8:]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), DES.block_size)
    return pt.decode()

# =============================
# 3. Server Code
# =============================
def server(port=12345, use_AES=True):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', port))
    s.listen(1)
    print(f"[Server] Listening on port {port}...")
    
    conn, addr = s.accept()
    print(f"[Server] Connection from {addr}")

    # Quantum-inspired key generation
    key_length = 16 if use_AES else 8
    key = generate_quantum_key(key_length)

    # Send key to client
    conn.send(key)

    # Bidirectional communication
    for _ in range(2):  # example: 2 message exchanges
        # Receive encrypted message
        encrypted_msg = conn.recv(4096)
        if use_AES:
            message = decrypt_message_AES(encrypted_msg, key)
        else:
            message = decrypt_message_DES(encrypted_msg, key)
        print(f"[Server] Received (decrypted): {message}")

        # Server sends reply
        reply = f"Server ACK: {message}"
        if use_AES:
            enc_reply = encrypt_message_AES(reply, key)
        else:
            enc_reply = encrypt_message_DES(reply, key)
        conn.send(enc_reply)

    conn.close()
    print("[Server] Connection closed.")

# =============================
# 4. Client Code
# =============================
def client(port=12345, messages=None, use_AES=True):
    if messages is None:
        messages = ["Hello Quantum Socket!", "Second message from client"]

    time.sleep(1)  # ensure server is ready
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(('localhost', port))

    key_length = 16 if use_AES else 8
    key = s.recv(key_length)

    for msg in messages:
        # Encrypt and send
        if use_AES:
            enc_msg = encrypt_message_AES(msg, key)
        else:
            enc_msg = encrypt_message_DES(msg, key)
        s.send(enc_msg)
        print(f"[Client] Sent: {msg}")

        # Receive reply
        enc_reply = s.recv(4096)
        if use_AES:
            reply = decrypt_message_AES(enc_reply, key)
        else:
            reply = decrypt_message_DES(enc_reply, key)
        print(f"[Client] Received (decrypted): {reply}")

    s.close()
    print("[Client] Connection closed.")

# =============================
# 5. Run Simulation in Jupyter
# =============================
use_AES = True  # Change to False for DES

server_thread = threading.Thread(target=server, args=(12345, use_AES))
client_thread = threading.Thread(target=client, args=(12345, ["Hello!", "How are you?"], use_AES))

server_thread.start()
client_thread.start()

server_thread.join()
client_thread.join()


[Server] Listening on port 12345...
[Server] Connection from ('127.0.0.1', 50615)
[Client] Sent: Hello!
[Server] Received (decrypted): Hello!
[Client] Received (decrypted): Server ACK: Hello!
[Client] Sent: How are you?
[Server] Received (decrypted): How are you?
[Server] Connection closed.
[Client] Received (decrypted): Server ACK: How are you?
[Client] Connection closed.


In [6]:
# =============================
# Interactive Hybrid Quantum-Classical Socket Simulation
# =============================

import socket
import threading
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np

def generate_quantum_key(length=16):
    qubits = np.random.randint(0, 2, size=length*8)
    key_bytes = bytearray()
    for i in range(0, len(qubits), 8):
        byte = 0
        for j in range(8):
            byte = (byte << 1) | qubits[i+j]
        key_bytes.append(byte)
    return bytes(key_bytes)

def encrypt_message_AES(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    return cipher.iv + ct_bytes

def decrypt_message_AES(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

# =============================
# Server
# =============================
def server(port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', port))
    s.listen(1)
    print("[Server] Listening...")
    
    conn, addr = s.accept()
    print(f"[Server] Connected by {addr}")

    key = generate_quantum_key(16)
    conn.send(key)  # send key to client

    while True:
        encrypted_msg = conn.recv(4096)
        if not encrypted_msg:
            break
        message = decrypt_message_AES(encrypted_msg, key)
        print(f"[Server] Client: {message}")

        if message.lower() == "quit":
            print("[Server] Client disconnected.")
            break

        reply = input("[Server] Your reply: ")
        enc_reply = encrypt_message_AES(reply, key)
        conn.send(enc_reply)

    conn.close()

# =============================
# Client
# =============================
def client(port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(('localhost', port))
    key = s.recv(16)  # receive AES key

    while True:
        msg = input("[Client] Type message: ")
        enc_msg = encrypt_message_AES(msg, key)
        s.send(enc_msg)

        if msg.lower() == "quit":
            print("[Client] Exiting...")
            break

        enc_reply = s.recv(4096)
        reply = decrypt_message_AES(enc_reply, key)
        print(f"[Client] Server: {reply}")

    s.close()

# =============================
# Run in Jupyter
# =============================
server_thread = threading.Thread(target=server)
client_thread = threading.Thread(target=client)

server_thread.start()
client_thread.start()

server_thread.join()
client_thread.join()


[Server] Listening...
[Server] Connected by ('127.0.0.1', 56427)


[Client] Type message:  hi 


[Server] Client: hi 


[Server] Your reply:  hello


[Client] Server: hello


[Client] Type message:  quit


[Server] Client: quit[Client] Exiting...

[Server] Client disconnected.


In [7]:
# =============================
# Interactive Hybrid Quantum-Classical Socket Simulation
# =============================

import socket
import threading
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np

# =============================
# 1. Quantum-inspired Key Generation
# =============================
def generate_quantum_key(length=16):
    qubits = np.random.randint(0, 2, size=length*8)
    key_bytes = bytearray()
    for i in range(0, len(qubits), 8):
        byte = 0
        for j in range(8):
            byte = (byte << 1) | qubits[i+j]
        key_bytes.append(byte)
    print(f"[Key] Generated quantum-inspired key ({length} bytes): {key_bytes.hex()}")
    return bytes(key_bytes)

# =============================
# 2. AES Encryption / Decryption with Logging
# =============================
def encrypt_message_AES(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    full_cipher = cipher.iv + ct_bytes
    print(f"[Encrypt] Message: '{message}'")
    print(f"[Encrypt] IV: {cipher.iv.hex()}")
    print(f"[Encrypt] Ciphertext: {ct_bytes.hex()}")
    return full_cipher

def decrypt_message_AES(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    print(f"[Decrypt] Received Ciphertext: {ct.hex()}")
    print(f"[Decrypt] IV: {iv.hex()}")
    print(f"[Decrypt] Decrypted Message: '{pt.decode()}'")
    return pt.decode()

# =============================
# 3. Server
# =============================
def server(port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', port))
    s.listen(1)
    print("[Server] Listening...")
    
    conn, addr = s.accept()
    print(f"[Server] Connected by {addr}")

    key = generate_quantum_key(16)
    conn.send(key)  # send key to client

    while True:
        encrypted_msg = conn.recv(4096)
        if not encrypted_msg:
            break
        message = decrypt_message_AES(encrypted_msg, key)
        print(f"[Server] Client: {message}")

        if message.lower() == "quit":
            print("[Server] Client disconnected.")
            break

        reply = input("[Server] Your reply: ")
        enc_reply = encrypt_message_AES(reply, key)
        conn.send(enc_reply)

    conn.close()

# =============================
# 4. Client
# =============================
def client(port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(('localhost', port))
    key = s.recv(16)  # receive AES key
    print(f"[Client] Received key: {key.hex()}")

    while True:
        msg = input("[Client] Type message: ")
        enc_msg = encrypt_message_AES(msg, key)
        s.send(enc_msg)

        if msg.lower() == "quit":
            print("[Client] Exiting...")
            break

        enc_reply = s.recv(4096)
        reply = decrypt_message_AES(enc_reply, key)
        print(f"[Client] Server: {reply}")

    s.close()

# =============================
# 5. Run Simulation
# =============================
server_thread = threading.Thread(target=server)
client_thread = threading.Thread(target=client)

server_thread.start()
client_thread.start()

server_thread.join()
client_thread.join()


[Server] Listening...
[Server] Connected by ('127.0.0.1', 57597)
[Key] Generated quantum-inspired key (16 bytes): b9318f3fef2b8ad564b50981fc9ea706
[Client] Received key: b9318f3fef2b8ad564b50981fc9ea706


[Client] Type message:  hi 


[Encrypt] Message: 'hi '
[Encrypt] IV: 6bf4a90eec87f9c754cea35eb8d50845
[Encrypt] Ciphertext: bb1262b814dd1af5d8409785899d6662
[Decrypt] Received Ciphertext: bb1262b814dd1af5d8409785899d6662
[Decrypt] IV: 6bf4a90eec87f9c754cea35eb8d50845
[Decrypt] Decrypted Message: 'hi '
[Server] Client: hi 


[Server] Your reply:  quit 


[Encrypt] Message: 'quit '
[Encrypt] IV: 4d4dc1e2291fa58484b8b85a14a92a6d
[Encrypt] Ciphertext: 2eec7f9673bf6241375fd6d5ad70e54c
[Decrypt] Received Ciphertext: 2eec7f9673bf6241375fd6d5ad70e54c
[Decrypt] IV: 4d4dc1e2291fa58484b8b85a14a92a6d
[Decrypt] Decrypted Message: 'quit '
[Client] Server: quit 


[Client] Type message:  quit


[Encrypt] Message: 'quit'
[Encrypt] IV: 727825e0a1da1f1400a2696fa6f36440
[Encrypt] Ciphertext: 900c1f0bb2d502b4142985b4abce24e9
[Decrypt] Received Ciphertext: 900c1f0bb2d502b4142985b4abce24e9
[Decrypt] IV: 727825e0a1da1f1400a2696fa6f36440
[Decrypt] Decrypted Message: 'quit'
[Server] Client: quit
[Server] Client disconnected.
[Client] Exiting...
